# March Machine Learning Mania
In this notebook, we:
- Try different models/features and assess their performance.
- Simulate multiple 2024 March Madness brackets.

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from gc import collect
import os
import sys
import tqdm

from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR, SVC
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.metrics import mean_squared_error, r2_score, log_loss, accuracy_score, confusion_matrix, classification_report
from xgboost import XGBRegressor, XGBClassifier

# display 100 rows and 100 columns
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 150)

# global random seed
SEED = 0

# set numpy seed
np.random.seed(SEED)

## Load Data

In [108]:
# load in features compact
fcomp = pd.read_csv('data/processed/features_compact.csv')

# load in features detailed
# fdet = pd.read_csv('data/processed/features_detailed.csv')

## Rearranging the Data
When we input a new row into our model (for the 2024 bracket), the winner could be the first or the second team in the row that we input into our model. Currently, all rows have the winning team on the left, and this will cause our model to learn this locational information. We need to rearrange the order of winners and losers, to where it is essentially random to prevent the model from learning this.

In [109]:
# # subtract starting yeaer from season col (to represent years since first tourney). this may capture changes in game play over time
fcomp['Season'] = np.where(fcomp['WTeamID'] < 3000, fcomp['Season'] - 1985, fcomp['Season'] - 1998)

# drop columns that can't be used as features
cols_def_drop = ['DayNum', 'WRegion', 'Wseed_diff', 'LRegion', 'Lseed_diff', 'abs_seed_diff']

# drop cols
fcomp = fcomp.drop(columns=(cols_def_drop))

In [110]:
# rename W, L to A, B
fcomp.columns = [x.replace('W', 'A_', 1) if x[0] == 'W' else x for x in fcomp.columns]
fcomp.columns = [x.replace('L', 'B_', 1) if x[0] == 'L' else x for x in fcomp.columns]

# check
fcomp.head()

,Season,A_TeamID,A_Score,B_TeamID,B_Score,A_Loc,NumOT,A_Seed,B_Seed,A_FullSeed,B_FullSeed,A_PlayIn,B_PlayIn,A_max_win_diff,A_max_loss_diff,A_num_games,A_win_ratio,A_mean_ppg,A_mean_papg,A_std_ppg,A_std_papg,A_mean_diff,A_std_diff,A_home_win_ratio,A_away_win_ratio,A_neutral_win_ratio,A_1_pos_win_missing,A_1_pos_loss_missing,A_ot_win_missing,A_ot_loss_missing,A_1_pos_game_ratio,A_1_pos_win_ratio,A_ot_ratio,A_ot_win_ratio,A_recent_win_ratio,A_recent_mean_pts_for,A_recent_mean_pts_against,A_recent_mean_score_diff,A_recent_std_pts_for,A_recent_std_pts_against,A_recent_std_score_diff,B_max_win_diff,B_max_loss_diff,B_num_games,B_win_ratio,B_mean_ppg,B_mean_papg,B_std_ppg,B_std_papg,B_mean_diff,B_std_diff,B_home_win_ratio,B_away_win_ratio,B_neutral_win_ratio,B_1_pos_win_missing,B_1_pos_loss_missing,B_ot_win_missing,B_ot_loss_missing,B_1_pos_game_ratio,B_1_pos_win_ratio,B_ot_ratio,B_ot_win_ratio,B_recent_win_ratio,B_recent_mean_pts_for,B_recent_mean_pts_against,B_recent_mean_score_diff,B_recent_std_pts_for,B_recent_std_pts_against,B_recent_std_score_diff,B_Loc,round,A_seed_win_prob,B_seed_win_prob
0,0,1116,63,1234,54,N,0,9,8,X09,X08,0,0,35,20,33,0.636364,65.333333,61.696970,11.332454,9.055902,3.636364,9.620253,0.909091,0.333333,0.700000,0,0,1,1,0.363636,0.250000,0.000000,0.0,0.666667,74.166667,61.500000,12.666667,15.382169,8.624899,8.286535,49,20,30,0.666667,69.733333,59.266667,12.375335,11.243712,10.466667,13.416676,0.833333,0.333333,0.666667,0,0,1,1,0.233333,0.142857,0.000000,0.0,0.333333,62.500000,63.833333,-1.333333,7.207249,15.625833,9.061518,N,1,0.452020,0.547980
1,0,1120,59,1345,58,N,0,11,6,Z11,Z06,0,0,42,19,29,0.620690,70.344828,66.655172,13.523340,11.519276,3.689655,12.768266,0.666667,0.454545,0.833333,0,0,0,0,0.482759,0.428571,0.068966,0.5,0.833333,63.166667,57.000000,6.166667,14.250731,6.689544,9.278110,31,43,25,0.680000,69.120000,65.320000,10.537755,8.837016,3.800000,13.685843,0.714286,0.600000,1.000000,0,0,1,1,0.400000,0.200000,0.000000,0.0,0.666667,66.333333,68.833333,-2.500000,12.270108,4.778424,11.172884,N,1,0.340909,0.659091
2,0,1207,68,1250,43,N,0,1,16,W01,W16,0,0,41,2,27,0.925926,75.740741,60.074074,11.475417,10.523929,15.666667,10.490513,0.923077,0.857143,1.000000,0,1,1,1,0.074074,1.000000,0.000000,0.0,1.000000,83.666667,64.333333,19.333333,10.366613,10.726913,8.594572,18,31,29,0.379310,65.758621,70.206897,8.131251,8.504503,-4.448276,16.284512,0.333333,0.363636,0.500000,0,0,0,1,0.724138,0.238095,0.034483,1.0,0.833333,69.500000,70.000000,-0.500000,7.593857,7.071068,1.500000,N,1,0.986842,0.013158
3,0,1229,58,1425,55,N,0,9,8,Y09,Y08,0,0,31,13,27,0.740741,71.592593,65.629630,10.506689,10.519915,5.962963,8.956982,0.785714,0.600000,1.000000,0,0,1,1,0.370370,0.400000,0.000000,0.0,0.666667,72.000000,64.833333,7.166667,9.006171,6.940221,8.363280,33,21,28,0.678571,68.392857,64.607143,9.272442,8.751871,3.785714,9.777817,0.642857,0.727273,0.666667,0,0,1,1,0.428571,0.500000,0.000000,0.0,0.500000,68.833333,68.166667,0.666667,6.879922,13.083068,11.561430,N,1,0.452020,0.547980
4,0,1242,49,1325,38,N,0,3,14,Z03,Z14,0,0,27,19,30,0.766667,76.033333,70.400000,10.420842,11.206870,5.633333,8.081707,1.000000,0.600000,0.500000,0,0,1,1,0.300000,0.333333,0.000000,0.0,0.833333,76.833333,72.166667,4.666667,7.461010,5.228129,5.416026,23,17,27,0.740741,67.555556,63.000000,9.638866,8.801914,4.555556,7.092059,1.000000,0.583333,0.600000,0,0,1,1,0.333333,0.333333,0.000000,0.0,0.833333,68.333333,58.166667,10.166667,7.455423,4.941322,7.762087,N,1,0.872093,0.127907


In [111]:
# choose (half) random rows to flip
np.random.seed(SEED)
flip = np.random.choice(fcomp.index, int(fcomp.shape[0] / 2), replace=False)

# flip A, B cols
for col in fcomp.columns:
    if col[0] == 'A':
        fcomp.loc[flip, col], fcomp.loc[flip, col.replace('A_', 'B_')] = fcomp.loc[flip, col.replace('A_', 'B_')].values, fcomp.loc[flip, col].values

# rearrange cols
df = fcomp.reindex(sorted(fcomp.columns), axis=1)

# check
df.head()

,A_1_pos_game_ratio,A_1_pos_loss_missing,A_1_pos_win_missing,A_1_pos_win_ratio,A_FullSeed,A_Loc,A_PlayIn,A_Score,A_Seed,A_TeamID,A_away_win_ratio,A_home_win_ratio,A_max_loss_diff,A_max_win_diff,A_mean_diff,A_mean_papg,A_mean_ppg,A_neutral_win_ratio,A_num_games,A_ot_loss_missing,A_ot_ratio,A_ot_win_missing,A_ot_win_ratio,A_recent_mean_pts_against,A_recent_mean_pts_for,A_recent_mean_score_diff,A_recent_std_pts_against,A_recent_std_pts_for,A_recent_std_score_diff,A_recent_win_ratio,A_seed_win_prob,A_std_diff,A_std_papg,A_std_ppg,A_win_ratio,B_1_pos_game_ratio,B_1_pos_loss_missing,B_1_pos_win_missing,B_1_pos_win_ratio,B_FullSeed,B_Loc,B_PlayIn,B_Score,B_Seed,B_TeamID,B_away_win_ratio,B_home_win_ratio,B_max_loss_diff,B_max_win_diff,B_mean_diff,B_mean_papg,B_mean_ppg,B_neutral_win_ratio,B_num_games,B_ot_loss_missing,B_ot_ratio,B_ot_win_missing,B_ot_win_ratio,B_recent_mean_pts_against,B_recent_mean_pts_for,B_recent_mean_score_diff,B_recent_std_pts_against,B_recent_std_pts_for,B_recent_std_score_diff,B_recent_win_ratio,B_seed_win_prob,B_std_diff,B_std_papg,B_std_ppg,B_win_ratio,NumOT,Season,round
0,0.363636,0,0,0.250000,X09,N,0,63,9,1116,0.333333,0.909091,20,35,3.636364,61.696970,65.333333,0.7,33,1,0.000000,1,0.0,61.500000,74.166667,12.666667,8.624899,15.382169,8.286535,0.666667,0.452020,9.620253,9.055902,11.332454,0.636364,0.233333,0,0,0.142857,X08,N,0,54,8,1234,0.333333,0.833333,20,49,10.466667,59.266667,69.733333,0.666667,30,1,0.000000,1,0.0,63.833333,62.500000,-1.333333,15.625833,7.207249,9.061518,0.333333,0.547980,13.416676,11.243712,12.375335,0.666667,0,0,1
1,0.400000,0,0,0.200000,Z06,N,0,58,6,1345,0.600000,0.714286,43,31,3.800000,65.320000,69.120000,1.0,25,1,0.000000,1,0.0,68.833333,66.333333,-2.500000,4.778424,12.270108,11.172884,0.666667,0.659091,13.685843,8.837016,10.537755,0.680000,0.482759,0,0,0.428571,Z11,N,0,59,11,1120,0.454545,0.666667,19,42,3.689655,66.655172,70.344828,0.833333,29,0,0.068966,0,0.5,57.000000,63.166667,6.166667,6.689544,14.250731,9.278110,0.833333,0.340909,12.768266,11.519276,13.523340,0.620690,0,0,1
2,0.724138,0,0,0.238095,W16,N,0,43,16,1250,0.363636,0.333333,31,18,-4.448276,70.206897,65.758621,0.5,29,1,0.034483,0,1.0,70.000000,69.500000,-0.500000,7.071068,7.593857,1.500000,0.833333,0.013158,16.284512,8.504503,8.131251,0.379310,0.074074,1,0,1.000000,W01,N,0,68,1,1207,0.857143,0.923077,2,41,15.666667,60.074074,75.740741,1.000000,27,1,0.000000,1,0.0,64.333333,83.666667,19.333333,10.726913,10.366613,8.594572,1.000000,0.986842,10.490513,10.523929,11.475417,0.925926,0,0,1
3,0.370370,0,0,0.400000,Y09,N,0,58,9,1229,0.600000,0.785714,13,31,5.962963,65.629630,71.592593,1.0,27,1,0.000000,1,0.0,64.833333,72.000000,7.166667,6.940221,9.006171,8.363280,0.666667,0.452020,8.956982,10.519915,10.506689,0.740741,0.428571,0,0,0.500000,Y08,N,0,55,8,1425,0.727273,0.642857,21,33,3.785714,64.607143,68.392857,0.666667,28,1,0.000000,1,0.0,68.166667,68.833333,0.666667,13.083068,6.879922,11.561430,0.500000,0.547980,9.777817,8.751871,9.272442,0.678571,0,0,1
4,0.333333,0,0,0.333333,Z14,N,0,38,14,1325,0.583333,1.000000,17,23,4.555556,63.000000,67.555556,0.6,27,1,0.000000,1,0.0,58.166667,68.333333,10.166667,4.941322,7.455423,7.762087,0.833333,0.127907,7.092059,8.801914,9.638866,0.740741,0.300000,0,0,0.333333,Z03,N,0,49,3,1242,0.600000,1.000000,19,27,5.633333,70.400000,76.033333,0.500000,30,1,0.000000,1,0.0,72.166667,76.833333,4.666667,5.228129,7.461010,5.416026,0.833333,0.872093,8.081707,11.206870,10.420842,0.766667,0,0,1


In [112]:
# one-hot encode A_Loc col, drop loc cols
fcomp = pd.get_dummies(fcomp, columns=['A_Loc'], dtype=int)
fcomp = fcomp.drop(columns=['A_Loc_N', 'B_Loc'])

# one-hot encode 'round' col, drop one col
fcomp = pd.get_dummies(fcomp, columns=['round'], dtype=int)
fcomp = fcomp.drop(columns=['round_1'])

## Create Labels

In [113]:
# create regression label, drop cols
fcomp['score_diff'] = fcomp['A_Score'] - fcomp['B_Score']
fcomp = fcomp.drop(columns=['A_Score', 'B_Score'])

# create binary label
fcomp['win'] = fcomp['score_diff'].apply(lambda x: 1 if x > 0 else 0)

# check
fcomp.head()

,Season,A_TeamID,B_TeamID,NumOT,A_Seed,B_Seed,A_FullSeed,B_FullSeed,A_PlayIn,B_PlayIn,A_max_win_diff,A_max_loss_diff,A_num_games,A_win_ratio,A_mean_ppg,A_mean_papg,A_std_ppg,A_std_papg,A_mean_diff,A_std_diff,A_home_win_ratio,A_away_win_ratio,A_neutral_win_ratio,A_1_pos_win_missing,A_1_pos_loss_missing,A_ot_win_missing,A_ot_loss_missing,A_1_pos_game_ratio,A_1_pos_win_ratio,A_ot_ratio,A_ot_win_ratio,A_recent_win_ratio,A_recent_mean_pts_for,A_recent_mean_pts_against,A_recent_mean_score_diff,A_recent_std_pts_for,A_recent_std_pts_against,A_recent_std_score_diff,B_max_win_diff,B_max_loss_diff,B_num_games,B_win_ratio,B_mean_ppg,B_mean_papg,B_std_ppg,B_std_papg,B_mean_diff,B_std_diff,B_home_win_ratio,B_away_win_ratio,B_neutral_win_ratio,B_1_pos_win_missing,B_1_pos_loss_missing,B_ot_win_missing,B_ot_loss_missing,B_1_pos_game_ratio,B_1_pos_win_ratio,B_ot_ratio,B_ot_win_ratio,B_recent_win_ratio,B_recent_mean_pts_for,B_recent_mean_pts_against,B_recent_mean_score_diff,B_recent_std_pts_for,B_recent_std_pts_against,B_recent_std_score_diff,A_seed_win_prob,B_seed_win_prob,A_Loc_A,A_Loc_H,round_2,round_3,round_4,round_5,round_6,score_diff,win
0,0,1116,1234,0,9,8,X09,X08,0,0,35,20,33,0.636364,65.333333,61.696970,11.332454,9.055902,3.636364,9.620253,0.909091,0.333333,0.7,0,0,1,1,0.363636,0.250000,0.000000,0.0,0.666667,74.166667,61.500000,12.666667,15.382169,8.624899,8.286535,49,20,30,0.666667,69.733333,59.266667,12.375335,11.243712,10.466667,13.416676,0.833333,0.333333,0.666667,0,0,1,1,0.233333,0.142857,0.000000,0.0,0.333333,62.500000,63.833333,-1.333333,7.207249,15.625833,9.061518,0.452020,0.547980,0,0,0,0,0,0,0,9,1
1,0,1345,1120,0,6,11,Z06,Z11,0,0,31,43,25,0.680000,69.120000,65.320000,10.537755,8.837016,3.800000,13.685843,0.714286,0.600000,1.0,0,0,1,1,0.400000,0.200000,0.000000,0.0,0.666667,66.333333,68.833333,-2.500000,12.270108,4.778424,11.172884,42,19,29,0.620690,70.344828,66.655172,13.523340,11.519276,3.689655,12.768266,0.666667,0.454545,0.833333,0,0,0,0,0.482759,0.428571,0.068966,0.5,0.833333,63.166667,57.000000,6.166667,14.250731,6.689544,9.278110,0.659091,0.340909,0,0,0,0,0,0,0,-1,0
2,0,1250,1207,0,16,1,W16,W01,0,0,18,31,29,0.379310,65.758621,70.206897,8.131251,8.504503,-4.448276,16.284512,0.333333,0.363636,0.5,0,0,0,1,0.724138,0.238095,0.034483,1.0,0.833333,69.500000,70.000000,-0.500000,7.593857,7.071068,1.500000,41,2,27,0.925926,75.740741,60.074074,11.475417,10.523929,15.666667,10.490513,0.923077,0.857143,1.000000,0,1,1,1,0.074074,1.000000,0.000000,0.0,1.000000,83.666667,64.333333,19.333333,10.366613,10.726913,8.594572,0.013158,0.986842,0,0,0,0,0,0,0,-25,0
3,0,1229,1425,0,9,8,Y09,Y08,0,0,31,13,27,0.740741,71.592593,65.629630,10.506689,10.519915,5.962963,8.956982,0.785714,0.600000,1.0,0,0,1,1,0.370370,0.400000,0.000000,0.0,0.666667,72.000000,64.833333,7.166667,9.006171,6.940221,8.363280,33,21,28,0.678571,68.392857,64.607143,9.272442,8.751871,3.785714,9.777817,0.642857,0.727273,0.666667,0,0,1,1,0.428571,0.500000,0.000000,0.0,0.500000,68.833333,68.166667,0.666667,6.879922,13.083068,11.561430,0.452020,0.547980,0,0,0,0,0,0,0,3,1
4,0,1325,1242,0,14,3,Z14,Z03,0,0,23,17,27,0.740741,67.555556,63.000000,9.638866,8.801914,4.555556,7.092059,1.000000,0.583333,0.6,0,0,1,1,0.333333,0.333333,0.000000,0.0,0.833333,68.333333,58.166667,10.166667,7.455423,4.941322,7.762087,27,19,30,0.766667,76.033333,70.400000,10.420842,11.206870,5.633333,8.081707,1.000000,0.600000,0.500000,0,0,1,1,0.300000,0.333333,0.000000,0.0,0.833333,76.833333,72.166667,4.666667,7.461010,5.228129,5.416026,0.127907,0.872093,0,0,0,0,0,0,0,-11,0


In [114]:
# create adjusted score diff col (score diff is inversely scaled by NumOT periods), drop NumOT
fcomp['score_diff_adj'] = fcomp['score_diff'] / (2 ** fcomp['NumOT'])

# check
fcomp[fcomp['NumOT'] > 0][['score_diff', 'NumOT', 'score_diff_adj']].sort_values(by='NumOT', ascending=False).head()

,score_diff,NumOT,score_diff_adj
654,-8,3,-1.00
1425,4,2,1.00
1676,7,2,1.75
1118,-4,2,-1.00
1166,1,2,0.25


__score_diff__ and __win__ can both be used as labels. They are both calculated with respect to team A.

In [115]:
# drop cols now that we have adjusted score diff
fcomp = fcomp.drop(columns=['score_diff', 'NumOT'])

In [116]:
# drop cols that may not be useful features
fcomp = fcomp.drop(columns=(['Season', 'A_Seed', 'B_Seed'])) # 'A_seed_win_prob', 'B_seed_win_prob', 

## Differenced Features
In combination with the team A/B data, we will also try training models using the differences between both teams' aggregated stats.

In [117]:
# create diff cols (features between 2 teams)
cols_to_diff = ['1_pos_game_ratio', '1_pos_win_ratio', 'away_win_ratio', 'home_win_ratio', 'max_loss_diff', 'max_win_diff', 'mean_diff', 'mean_papg', 'mean_ppg', 
                'neutral_win_ratio', 'num_games', 'ot_ratio', 'ot_win_ratio', 'recent_mean_pts_against', 'recent_mean_pts_for', 'recent_mean_score_diff', 'recent_std_pts_against', 
                'recent_std_pts_for', 'recent_std_score_diff', 'recent_win_ratio', 'std_diff', 'std_papg', 'std_ppg', 'win_ratio', 'seed_win_prob']

# features to be kept but shouldn't be differenced
cols_to_keep = ['A_TeamID', 'A_FullSeed', 'A_1_pos_loss_missing', 'A_1_pos_win_missing', 'A_ot_loss_missing', 'A_ot_win_missing', 'A_PlayIn', 'B_TeamID', 'B_FullSeed', 
                'B_1_pos_loss_missing', 'B_1_pos_win_missing', 'B_ot_loss_missing', 'B_ot_win_missing', 'B_PlayIn', 'A_Loc_A', 'A_Loc_H', 'round_2', 'round_3', 'round_4', 'round_5', 
                'round_6', 'score_diff_adj', 'win']

# create diff df
fcomp_diff = fcomp[cols_to_keep].copy()

# create diff cols
for col in cols_to_diff:
    fcomp_diff[col + '_diff'] = fcomp['A_' + col] - fcomp['B_' + col]

# check
fcomp_diff.head()

,A_TeamID,A_FullSeed,A_1_pos_loss_missing,A_1_pos_win_missing,A_ot_loss_missing,A_ot_win_missing,A_PlayIn,B_TeamID,B_FullSeed,B_1_pos_loss_missing,B_1_pos_win_missing,B_ot_loss_missing,B_ot_win_missing,B_PlayIn,A_Loc_A,A_Loc_H,round_2,round_3,round_4,round_5,round_6,score_diff_adj,win,1_pos_game_ratio_diff,1_pos_win_ratio_diff,away_win_ratio_diff,home_win_ratio_diff,max_loss_diff_diff,max_win_diff_diff,mean_diff_diff,mean_papg_diff,mean_ppg_diff,neutral_win_ratio_diff,num_games_diff,ot_ratio_diff,ot_win_ratio_diff,recent_mean_pts_against_diff,recent_mean_pts_for_diff,recent_mean_score_diff_diff,recent_std_pts_against_diff,recent_std_pts_for_diff,recent_std_score_diff_diff,recent_win_ratio_diff,std_diff_diff,std_papg_diff,std_ppg_diff,win_ratio_diff,seed_win_prob_diff
0,1116,X09,0,0,1,1,0,1234,X08,0,0,1,1,0,0,0,0,0,0,0,0,9.0,1,0.130303,0.107143,0.000000,0.075758,0,-14,-6.830303,2.430303,-4.400000,0.033333,3,0.000000,0.0,-2.333333,11.666667,14.000000,-7.000934,8.174919,-0.774983,0.333333,-3.796423,-2.187809,-1.042881,-0.030303,-0.095960
1,1345,Z06,0,0,1,1,0,1120,Z11,0,0,0,0,0,0,0,0,0,0,0,0,-1.0,0,-0.082759,-0.228571,0.145455,0.047619,24,-11,0.110345,-1.335172,-1.224828,0.166667,-4,-0.068966,-0.5,11.833333,3.166667,-8.666667,-1.911120,-1.980623,1.894773,-0.166667,0.917577,-2.682259,-2.985585,0.059310,0.318182
2,1250,W16,0,0,1,0,0,1207,W01,1,0,1,1,0,0,0,0,0,0,0,0,-25.0,0,0.650064,-0.761905,-0.493506,-0.589744,29,-23,-20.114943,10.132822,-9.982120,-0.500000,2,0.034483,1.0,5.666667,-14.166667,-19.833333,-3.655845,-2.772756,-7.094572,-0.166667,5.793999,-2.019426,-3.344166,-0.546616,-0.973684
3,1229,Y09,0,0,1,1,0,1425,Y08,0,0,1,1,0,0,0,0,0,0,0,0,3.0,1,-0.058201,-0.100000,-0.127273,0.142857,-8,-2,2.177249,1.022487,3.199735,0.333333,-1,0.000000,0.0,-3.333333,3.166667,6.500000,-6.142847,2.126248,-3.198151,0.166667,-0.820835,1.768045,1.234247,0.062169,-0.095960
4,1325,Z14,0,0,1,1,0,1242,Z03,0,0,1,1,0,0,0,0,0,0,0,0,-11.0,0,0.033333,0.000000,-0.016667,0.000000,-2,-4,-1.077778,-7.400000,-8.477778,0.100000,-3,0.000000,0.0,-14.000000,-8.500000,5.500000,-0.286807,-0.005587,2.346062,0.000000,-0.989648,-2.404957,-0.781976,-0.025926,-0.744186


## Split Data

In [118]:
# split on gender, drop cols
mfcomp = fcomp[fcomp['A_TeamID'] < 3000]
wfcomp = fcomp[fcomp['A_TeamID'] >= 3000]

# same for diff data
mfcomp_diff = fcomp_diff[fcomp_diff['A_TeamID'] < 3000]
wfcomp_diff = fcomp_diff[fcomp_diff['A_TeamID'] >= 3000]

## Chalk Bracket
Here, we will simply predict the better seed. If seeds are equal (in rounds 5 and 6), we will predict the team with the better win_ratio.

From the 38 NCAA Men's tournaments that we have access to, the better seed wins __70.62%__ of the time. In the women's, the better seed wins __76.51%__ of the time. In conjunction to predicting the better seed (when seeds are different), we will predict a full __chalk__ bracket by taking the team with a higher regular season win ratio when the seeds are the same.

In [12]:
def get_dummy_preds(data):
    """
    Function to get dummy predictions based on seed and win percentage
    """

    # create a container
    dummy_preds = []

    # loop through the dataframe based on conditions
    for idx, row in data.iterrows():
        if data.loc[idx, "A_Seed"] < data.loc[idx, "B_Seed"]:
            dummy_preds.append(1)
        elif data.loc[idx, "A_Seed"] > data.loc[idx, "B_Seed"]:
            dummy_preds.append(0)
        else:
            if data.loc[idx, "A_win_ratio"] > data.loc[idx, "B_win_ratio"]:
                dummy_preds.append(1)
            else:
                dummy_preds.append(0)
    
    return np.array(dummy_preds)

### Men's

In [13]:
# get dummy preds
mchalk_preds = get_dummy_preds(mfcomp)

# compare preds to win col
mchalk_acc = accuracy_score(mfcomp['win'], mchalk_preds)

print(f"Accuracy of dummy predictions: {mchalk_acc*100:.2f}%.")

Accuracy of dummy predictions: 71.04%.


### Women's

In [14]:
# get dummy preds
wchalk_preds = get_dummy_preds(wfcomp)

# compare preds to win col
wchalk_acc = accuracy_score(wfcomp['win'], wchalk_preds)

print(f"Accuracy of dummy predictions: {wchalk_acc*100:.2f}%")

Accuracy of dummy predictions: 77.65%


## Regression/Classification
This problem can be treated as a regression problem (label is __score differential__). It can also be treated as binary classification (label is __win/loss__).

We will try the original aggregated stats for both team A and B as features (mfcomp/wfcomp), and also the differences between the two teams' stats (mcomp_diff/wcomp_diff).

In [119]:
# non feature cols
non_feat_cols = ['A_TeamID', 'A_FullSeed', 'B_TeamID', 'B_FullSeed', 'win', 'score_diff_adj']

# look at num feats for the 2 datasets
print(f'Features of both team A and B: {mfcomp.shape[1] - len(non_feat_cols)}')
print(f'Features of the DIFFERENCE between team A and B: {mfcomp_diff.shape[1] - len(non_feat_cols)}')

Features of both team A and B: 67
Features of the DIFFERENCE between team A and B: 42


### Data Preprocessing

In [120]:
# create feature order list so our models are trained on the same feature order
sorted_cols = sorted(fcomp.columns)
sorted_cols.remove('A_Loc_A')
sorted_cols.remove('A_Loc_H')
sorted_cols.remove('score_diff_adj')
sorted_cols.remove('win')
sorted_cols.extend(['A_Loc_A', 'A_Loc_H', 'score_diff_adj', 'win'])

# same for diff
diff_cols = sorted(fcomp_diff.columns)
diff_cols.remove('A_Loc_A')
diff_cols.remove('A_Loc_H')
diff_cols.remove('score_diff_adj')
diff_cols.remove('win')
diff_cols.extend(['A_Loc_A', 'A_Loc_H', 'score_diff_adj', 'win'])

In [121]:
# function that takes in estimator, data, models_df, tournament, test_size
def run_model(estimator, features, regression, models_df, tournament, folds=5):
    """
    Run a model on data and save results to models_df.

    Parameters
    ----------
    estimator : sklearn estimator
        Estimator to use for modeling.
    data : pd.DataFrame
        Data to model.
    regression : bool
        Whether to model score differential (regression) or win/loss (classification).
    models_df : pd.DataFrame
        DataFrame to save results to.
    tournament : str
        Gender - 'M' or 'F'.
    folds : int
        Number of cross-validation folds to use.

    Returns
    -------
    models_df.tail() : pd.DataFrame
        Last 5 rows of dataframe with results from model.
    """
    
    # create copy to avoid modification
    data = features.copy()

    # define which col order to use
    if data.shape[1] == 73:
        data = data[sorted_cols]
    elif data.shape[1] == 48:
        data = data[diff_cols]
    else:
        print(f'Number of features: {data.shape[1]}')
        raise ValueError('Invalid number of features.')

    # drop unused cols
    data = data.drop(columns=['A_TeamID', 'B_TeamID', 'A_FullSeed', 'B_FullSeed'])

    # define cross-validation
    kf = KFold(n_splits=folds, shuffle=True, random_state=SEED)

    # create scaler
    scaler = MinMaxScaler()

    # predicting score differential
    if regression:
        # define X and y
        X = data.drop(columns=['score_diff_adj', 'win'])
        y = data['score_diff_adj']

        # initialize lists to store metrics
        rmse_scores_train = []
        rmse_scores_test = []
        r2_scores_train = []
        r2_scores_test = []
        acc_scores_train = []
        acc_scores_test = []
        r1_accs = []
        r2_accs = []
        r3_accs = []
        r4_accs = []
        r5_accs = []
        r6_accs = []

        for train_index, test_index in kf.split(X):
            # split data
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            # scale data
            X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
            X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

            # fit model
            estimator.fit(X_train_scaled, y_train)

            # predict
            train_preds = estimator.predict(X_train_scaled)
            test_preds = pd.Series(estimator.predict(X_test_scaled), index=y_test.index)

            # evaluate
            rmse_scores_train.append(mean_squared_error(y_train, train_preds, squared=False))
            rmse_scores_test.append(mean_squared_error(y_test, test_preds, squared=False))
            r2_scores_train.append(r2_score(y_train, train_preds))
            r2_scores_test.append(r2_score(y_test, test_preds))
            acc_scores_train.append(accuracy_score(np.sign(y_train), np.sign(train_preds)))
            acc_scores_test.append(accuracy_score(np.sign(y_test), np.sign(test_preds)))

            # get indices of each round
            r1_idx = y_test.index[(data.loc[y_test.index, 'round_2'] == 0) & (data.loc[y_test.index, 'round_3'] == 0) & (data.loc[y_test.index, 'round_4'] == 0) & \
                                  (data.loc[y_test.index, 'round_5'] == 0) & (data.loc[y_test.index, 'round_6'] == 0)]
            r2_idx = y_test.index[data.loc[y_test.index, 'round_2'] == 1]
            r3_idx = y_test.index[data.loc[y_test.index, 'round_3'] == 1]
            r4_idx = y_test.index[data.loc[y_test.index, 'round_4'] == 1]
            r5_idx = y_test.index[data.loc[y_test.index, 'round_5'] == 1]
            r6_idx = y_test.index[data.loc[y_test.index, 'round_6'] == 1]

            # get accuracy of each round
            r1_accs.append(accuracy_score(np.sign(y_test.loc[r1_idx]), np.sign(test_preds.loc[r1_idx])))
            r2_accs.append(accuracy_score(np.sign(y_test.loc[r2_idx]), np.sign(test_preds.loc[r2_idx])))
            r3_accs.append(accuracy_score(np.sign(y_test.loc[r3_idx]), np.sign(test_preds.loc[r3_idx])))
            r4_accs.append(accuracy_score(np.sign(y_test.loc[r4_idx]), np.sign(test_preds.loc[r4_idx])))
            r5_accs.append(accuracy_score(np.sign(y_test.loc[r5_idx]), np.sign(test_preds.loc[r5_idx])))
            r6_accs.append(accuracy_score(np.sign(y_test.loc[r6_idx]), np.sign(test_preds.loc[r6_idx])))

        # average the metrics across folds
        train_rmse = np.mean(rmse_scores_train)
        test_rmse = np.mean(rmse_scores_test)
        train_r2 = np.mean(r2_scores_train)
        test_r2 = np.mean(r2_scores_test)
        train_acc = np.mean(acc_scores_train)
        test_acc = np.mean(acc_scores_test)
        r1_acc = np.mean(r1_accs)
        r2_acc = np.mean(r2_accs)
        r3_acc = np.mean(r3_accs)
        r4_acc = np.mean(r4_accs)
        r5_acc = np.mean(r5_accs)
        r6_acc = np.mean(r6_accs)
        
        # Save results to models_df
        models_df.loc[len(models_df.index)] = [tournament, 'A_adj_score_diff', estimator, X.shape[1], X.columns.to_list(), folds, train_r2, test_r2, train_rmse, test_rmse, train_acc, test_acc, r1_acc, r2_acc, r3_acc, r4_acc, r5_acc, r6_acc]

        # delete variables
        del X, y, kf, scaler, train_index, test_index, X_train, X_test, y_train, y_test, train_preds, test_preds, rmse_scores_train, rmse_scores_test, r2_scores_train, r2_scores_test, acc_scores_train, acc_scores_test, r1_idx, r2_idx, r3_idx, \
        r4_idx, r5_idx, r6_idx, r1_accs, r2_accs, r3_accs, r4_accs, r5_accs, r6_accs, train_rmse, test_rmse, train_r2, test_r2, train_acc, test_acc, r1_acc, r2_acc, r3_acc, r4_acc, r5_acc, r6_acc
        collect()
        
        return models_df.tail()
    
    # predicting win/loss
    else:
        # define X and y
        X = data.drop(columns=['score_diff_adj', 'win'])
        y = data['win']

        # initialize lists to store metrics
        log_loss_train = []
        log_loss_test = []
        acc_scores_train = []
        acc_scores_test = []
        r1_accs = []
        r2_accs = []
        r3_accs = []
        r4_accs = []
        r5_accs = []
        r6_accs = []

        for train_index, test_index in kf.split(X):
            # split data
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            # scale data
            X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
            X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

            # fit model
            estimator.fit(X_train_scaled, y_train)

            # predict
            train_preds = estimator.predict(X_train_scaled)
            test_preds = pd.Series(estimator.predict(X_test_scaled), index=y_test.index)


            # evaluate
            log_loss_train.append(log_loss(y_train, train_preds))
            log_loss_test.append(log_loss(y_test, test_preds))
            acc_scores_train.append(accuracy_score(np.sign(y_train), np.sign(train_preds)))
            acc_scores_test.append(accuracy_score(np.sign(y_test), np.sign(test_preds)))

            # get indices of each round
            r1_idx = y_test.index[(data.loc[y_test.index, 'round_2'] == 0) & (data.loc[y_test.index, 'round_3'] == 0) & (data.loc[y_test.index, 'round_4'] == 0) & \
                                  (data.loc[y_test.index, 'round_5'] == 0) & (data.loc[y_test.index, 'round_6'] == 0)]
            r2_idx = y_test.index[data.loc[y_test.index, 'round_2'] == 1]
            r3_idx = y_test.index[data.loc[y_test.index, 'round_3'] == 1]
            r4_idx = y_test.index[data.loc[y_test.index, 'round_4'] == 1]
            r5_idx = y_test.index[data.loc[y_test.index, 'round_5'] == 1]
            r6_idx = y_test.index[data.loc[y_test.index, 'round_6'] == 1]

            # get accuracy of each round
            r1_accs.append(accuracy_score(np.sign(y_test.loc[r1_idx]), np.sign(test_preds.loc[r1_idx])))
            r2_accs.append(accuracy_score(np.sign(y_test.loc[r2_idx]), np.sign(test_preds.loc[r2_idx])))
            r3_accs.append(accuracy_score(np.sign(y_test.loc[r3_idx]), np.sign(test_preds.loc[r3_idx])))
            r4_accs.append(accuracy_score(np.sign(y_test.loc[r4_idx]), np.sign(test_preds.loc[r4_idx])))
            r5_accs.append(accuracy_score(np.sign(y_test.loc[r5_idx]), np.sign(test_preds.loc[r5_idx])))
            r6_accs.append(accuracy_score(np.sign(y_test.loc[r6_idx]), np.sign(test_preds.loc[r6_idx])))

        # average the metrics across folds
        train_log_loss = np.mean(log_loss_train)
        test_log_loss = np.mean(log_loss_test)
        train_acc = np.mean(acc_scores_train)
        test_acc = np.mean(acc_scores_test)
        r1_acc = np.mean(r1_accs)
        r2_acc = np.mean(r2_accs)
        r3_acc = np.mean(r3_accs)
        r4_acc = np.mean(r4_accs)
        r5_acc = np.mean(r5_accs)
        r6_acc = np.mean(r6_accs)
        
        # save results to models_df
        models_df.loc[len(models_df.index)] = [tournament, 'A_Win', estimator, X.shape[1], X.columns.to_list(), 5, train_log_loss, test_log_loss, train_acc, test_acc, r1_acc, r2_acc, r3_acc, r4_acc, r5_acc, r6_acc]

        # delete variables
        del X, y, kf, scaler, train_index, test_index, X_train, X_test, y_train, y_test, train_preds, test_preds, log_loss_train, log_loss_test, acc_scores_train, acc_scores_test, r1_idx, r2_idx, r3_idx, \
        r4_idx, r5_idx, r6_idx, r1_accs, r2_accs, r3_accs, r4_accs, r5_accs, r6_accs, train_log_loss, test_log_loss, train_acc, test_acc, r1_acc, r2_acc, r3_acc, r4_acc, r5_acc, r6_acc
        collect()
        
        return models_df.tail()

### Regression (Predicting <ins>Score Differential</ins>)

#### Men's

In [18]:
# create a df to hold regression models
# reg_df = pd.DataFrame(columns=['Tournament', 'Label', 'Model', 'Num_Features', 'Features', 'Num_CV_Folds', 'Train_R2', 'Val_R2', 'Train_RMSE', 'Val_RMSE', 'Train_Acc', 'Val_Acc', 'Val_r1_acc', 'Val_r2_acc', 'Val_r3_acc', 'Val_r4_acc', 'Val_r5_acc', 'Val_r6_acc'])

# load reg df
reg_df = pd.read_csv('models/regression_models.csv')

In [122]:
# define regression models
models = [LinearRegression(n_jobs=-1), RandomForestRegressor(n_jobs=-1), XGBRegressor(n_jobs=-1), SVR(), KNeighborsRegressor(n_jobs=-1)]
datasets = [mfcomp] # , mfcomp_diff

# run regression models
for model in models:
    for dataset in datasets:
        # run model
        run_model(estimator=model, features=dataset, regression=True, models_df=reg_df, tournament='M')

In [123]:
# inspect
reg_df[reg_df['Tournament'] == 'M'].sort_values(by='Val_RMSE').head()

,Tournament,Label,Model,Num_Features,Features,Num_CV_Folds,Train_R2,Val_R2,Train_RMSE,Val_RMSE,Train_Acc,Val_Acc,Val_r1_acc,Val_r2_acc,Val_r3_acc,Val_r4_acc,Val_r5_acc,Val_r6_acc
41,M,A_adj_score_diff,LinearRegression(n_jobs=-1),42,"[1_pos_game_ratio_diff, 1_pos_win_ratio_diff, ...",5,0.420918,0.239967,11.131986,11.690725,0.718032,0.694111,0.733042,0.680732,0.677630,0.561640,0.577307,0.614603
61,M,A_adj_score_diff,LinearRegression(n_jobs=-1),43,"[1_pos_game_ratio_diff, 1_pos_win_ratio_diff, ...",5,0.421229,0.239849,11.129023,11.691471,0.719286,0.694950,0.736250,0.678716,0.673634,0.553640,0.588119,0.614603
80,M,A_adj_score_diff,LinearRegression(n_jobs=-1),67,"[A_1_pos_game_ratio, A_1_pos_loss_missing, A_1...",5,0.429856,0.239511,11.045665,11.701572,0.724928,0.704560,0.742861,0.690977,0.684622,0.560981,0.612601,0.636825
21,M,A_adj_score_diff,LinearRegression(n_jobs=-1),43,"[1_pos_game_ratio_diff, 1_pos_win_ratio_diff, ...",5,0.421932,0.237896,11.122295,11.706679,0.718868,0.697455,0.735508,0.681544,0.680528,0.566186,0.619841,0.609841
1,M,A_adj_score_diff,LinearRegression(n_jobs=-1),44,"[1_pos_game_ratio_diff, 1_pos_win_ratio_diff, ...",5,0.422245,0.237746,11.119313,11.707705,0.720749,0.695368,0.734541,0.683025,0.673717,0.561640,0.590170,0.609841


#### Women's

In [124]:
# define regression models
models = [LinearRegression(n_jobs=-1), RandomForestRegressor(n_jobs=-1), XGBRegressor(n_jobs=-1), SVR(), KNeighborsRegressor(n_jobs=-1)]
datasets = [wfcomp] # , wfcomp_diff

# run regression models
for model in models:
    for dataset in datasets:
        # run model
        run_model(estimator=model, features=dataset, regression=True, models_df=reg_df, tournament='W')

In [125]:
# inspect
reg_df[reg_df['Tournament'] == 'W'].sort_values(by='Val_RMSE').head()

,Tournament,Label,Model,Num_Features,Features,Num_CV_Folds,Train_R2,Val_R2,Train_RMSE,Val_RMSE,Train_Acc,Val_Acc,Val_r1_acc,Val_r2_acc,Val_r3_acc,Val_r4_acc,Val_r5_acc,Val_r6_acc
11,W,A_adj_score_diff,LinearRegression(n_jobs=-1),44,"[1_pos_game_ratio_diff, 1_pos_win_ratio_diff, ...",5,0.665216,0.641316,12.061677,12.453760,0.795556,0.787302,0.817503,0.786098,0.739216,0.747515,0.559652,0.885714
71,W,A_adj_score_diff,LinearRegression(n_jobs=-1),43,"[1_pos_game_ratio_diff, 1_pos_win_ratio_diff, ...",5,0.663259,0.640459,12.096787,12.467196,0.795873,0.786032,0.815118,0.780969,0.751395,0.759152,0.559652,0.857143
31,W,A_adj_score_diff,LinearRegression(n_jobs=-1),43,"[1_pos_game_ratio_diff, 1_pos_win_ratio_diff, ...",5,0.660717,0.637045,12.142411,12.526675,0.797937,0.788571,0.815049,0.788507,0.764471,0.738041,0.534652,0.885714
51,W,A_adj_score_diff,LinearRegression(n_jobs=-1),42,"[1_pos_game_ratio_diff, 1_pos_win_ratio_diff, ...",5,0.658645,0.635977,12.179324,12.543558,0.795873,0.787302,0.816351,0.776318,0.765354,0.759678,0.559652,0.857143
70,W,A_adj_score_diff,LinearRegression(n_jobs=-1),69,"[A_1_pos_game_ratio, A_1_pos_loss_missing, A_1...",5,0.670684,0.634342,11.962705,12.573288,0.800476,0.790476,0.824023,0.785752,0.745282,0.738041,0.611703,0.800000


In [126]:
# save reg_df
reg_df.to_csv('models/regression_models.csv', index=False)

### Classification (Predicting <ins>Win/Loss</ins>)

In [25]:
# create a df to hold classification models
# class_df = pd.DataFrame(columns=['Tournament', 'Label', 'Model', 'Num_Features', 'Features', 'Num_CV_Folds', 'Train_LogLoss', 'Val_LogLoss', 'Train_Acc', 'Val_Acc', 'Val_r1_acc', 'Val_r2_acc', 'Val_r3_acc', 'Val_r4_acc', 'Val_r5_acc', 'Val_r6_acc'])

# load class df
class_df = pd.read_csv('models/classification_models.csv')

#### Men's

In [127]:
# define regression models
models = [LogisticRegression(n_jobs=-1, random_state=SEED), RandomForestClassifier(n_jobs=-1), XGBClassifier(n_jobs=-1), SVC(), KNeighborsClassifier(n_jobs=-1)]
datasets = [mfcomp] # , mfcomp_diff

# run regression models
for model in models:
    for dataset in datasets:
        # run model
        run_model(estimator=model, features=dataset, regression=False, models_df=class_df, tournament='M')

In [128]:
# inspect
class_df[class_df['Tournament'] == 'M'].sort_values(by='Val_LogLoss').head()

,Tournament,Label,Model,Num_Features,Features,Num_CV_Folds,Train_LogLoss,Val_LogLoss,Train_Acc,Val_Acc,Val_r1_acc,Val_r2_acc,Val_r3_acc,Val_r4_acc,Val_r5_acc,Val_r6_acc
61,M,A_Win,"LogisticRegression(n_jobs=-1, random_state=0)",43,"[1_pos_game_ratio_diff, 1_pos_win_ratio_diff, ...",5,9.634281,11.171669,0.721062,0.676551,0.713051,0.665861,0.666748,0.518799,0.573833,0.602857
60,M,A_Win,"LogisticRegression(n_jobs=-1, random_state=0)",69,"[A_1_pos_game_ratio, A_1_pos_loss_missing, A_1...",5,9.349238,11.200459,0.729315,0.675717,0.712034,0.658136,0.671106,0.515847,0.620263,0.540952
80,M,A_Win,"LogisticRegression(n_jobs=-1, random_state=0)",67,"[A_1_pos_game_ratio, A_1_pos_loss_missing, A_1...",5,9.374504,11.229300,0.728584,0.674882,0.706296,0.666351,0.672186,0.523847,0.605977,0.540952
40,M,A_Win,"LogisticRegression(n_jobs=-1, random_state=0)",67,"[A_1_pos_game_ratio, A_1_pos_loss_missing, A_1...",5,9.323986,11.272685,0.730046,0.673626,0.709680,0.659617,0.671844,0.498756,0.608498,0.540952
1,M,A_Win,"LogisticRegression(n_jobs=-1, random_state=0)",44,"[1_pos_game_ratio_diff, 1_pos_win_ratio_diff, ...",5,9.551290,11.272741,0.723465,0.673624,0.708419,0.669892,0.669881,0.496405,0.543064,0.602857


#### Women's

In [129]:
# define regression models
models = [LogisticRegression(n_jobs=-1, random_state=SEED), RandomForestClassifier(n_jobs=-1), XGBClassifier(n_jobs=-1), SVC(), KNeighborsClassifier(n_jobs=-1)]
datasets = [wfcomp] # , wfcomp_diff

# run regression models
for model in models:
    for dataset in datasets:
        # run model
        run_model(estimator=model, features=dataset, regression=False, models_df=class_df, tournament='W')

In [130]:
# inspect
class_df[class_df['Tournament'] == 'W'].sort_values(by='Val_LogLoss').head()

,Tournament,Label,Model,Num_Features,Features,Num_CV_Folds,Train_LogLoss,Val_LogLoss,Train_Acc,Val_Acc,Val_r1_acc,Val_r2_acc,Val_r3_acc,Val_r4_acc,Val_r5_acc,Val_r6_acc
11,W,A_Win,"LogisticRegression(n_jobs=-1, random_state=0)",44,"[1_pos_game_ratio_diff, 1_pos_win_ratio_diff, ...",5,6.798191,7.192921,0.803175,0.791746,0.822894,0.772918,0.772548,0.769737,0.585421,0.72381
31,W,A_Win,"LogisticRegression(n_jobs=-1, random_state=0)",43,"[1_pos_game_ratio_diff, 1_pos_win_ratio_diff, ...",5,6.831084,7.236779,0.802222,0.790476,0.822733,0.774986,0.760211,0.769737,0.580421,0.72381
51,W,A_Win,"LogisticRegression(n_jobs=-1, random_state=0)",42,"[1_pos_game_ratio_diff, 1_pos_win_ratio_diff, ...",5,6.896873,7.258710,0.800317,0.789841,0.821597,0.767977,0.758438,0.789152,0.610421,0.72381
71,W,A_Win,"LogisticRegression(n_jobs=-1, random_state=0)",43,"[1_pos_game_ratio_diff, 1_pos_win_ratio_diff, ...",5,6.935251,7.302569,0.799206,0.788571,0.824316,0.767925,0.742794,0.780263,0.610421,0.72381
85,W,A_Win,"LogisticRegression(n_jobs=-1, random_state=0)",67,"[A_1_pos_game_ratio, A_1_pos_loss_missing, A_1...",5,6.831084,7.390288,0.802222,0.786032,0.821650,0.776300,0.728508,0.767018,0.595037,0.72381


In [131]:
# save class_df
class_df.to_csv('models/classification_models.csv', index=False)

- After manual model inspection, Linear Regression performed the best when we replaced the seed with seed_win_prob and used the team A/B data.

## Model Tuning
Now, we'll look at the best performing models for both the men's and women's bracket so that we can make 2024 predictions.

## Simultate N Brackets

In [136]:
# root dirs
root = 'data/'
mroot = 'data/mens/'
wroot = 'data/womens/'

# load data, get slots for 2024, drop play-ins
slots = pd.read_csv(mroot + 'MNCAATourneySlots.csv')
slots = slots[slots['Season'] == 2024]
slots = slots[slots['Slot'].str.contains('R')].reset_index(drop=True)

# load seed data
seeds_2024 = pd.read_csv(root + '2024_tourney_seeds.csv')

# load in 2024 data
df_2024 = pd.read_csv(root + 'processed/2024_features.csv').drop(columns=['Region', 'Season'])

# drop teams not in 2024 tourney
df_2024 = df_2024[df_2024['TeamID'].isin(seeds_2024['TeamID'])]

# drop play in char
df_2024['FullSeed'] = df_2024['FullSeed'].apply(lambda x: x[:3])

# split into mens and womens
mdf_2024 = df_2024[df_2024['TeamID'] < 3000].reset_index(drop=True)
wdf_2024 = df_2024[df_2024['TeamID'] >= 3000].reset_index(drop=True)

In [138]:
# men's historical seed differential winning percentages
mens_seed_win = [(-15, 0.013157894736842105),(-14, 0.04276315789473684),(-13, 0.07236842105263158),(-12, 0.10013769889840882),(-11, 0.12790697674418605),(-10, 0.16451214758997013),(-9, 0.2011173184357542),(-8, 0.25),(-7, 0.30364372469635625),(-6, 0.3222764078027236), 
                 (-5, 0.3409090909090909),(-4, 0.3565744192715802),(-3, 0.3722397476340694),(-2, 0.41212997482713576),(-1, 0.45202020202020204),(0, 0.5),(1, 0.547979797979798),(2, 0.5878700251728642),(3, 0.6277602523659306),(4, 0.6434255807284198),(5, 0.6590909090909092), 
                 (6, 0.6777235921972764),(7, 0.6963562753036437),(8, 0.75),(9, 0.7988826815642458),(10, 0.8354878524100299),(11, 0.872093023255814),(12, 0.8998623011015912),(13, 0.9276315789473684),(14, 0.9572368421052632),(15, 0.9868421052631579)]

# women's historical seed differential winning percentages
womens_seed_win = [(-15, 0.01),(-14, 0.017182890855457225),(-13, 0.024365781710914452),(-12, 0.03154867256637168),(-11, 0.0387315634218289),(-10, 0.04591445427728613),(-9, 0.05309734513274336),(-8, 0.17006802721088435),(-7, 0.17197452229299362),(-6, 0.22679185884764624), 
                   (-5, 0.28160919540229884),(-4, 0.30250672536072387),(-3, 0.32340425531914896),(-2, 0.3844744048872972),(-1, 0.44554455445544555),(0, 0.5),(1, 0.5544554455445545),(2, 0.6155255951127028),(3, 0.676595744680851),(4, 0.6974932746392761),(5, 0.7183908045977012), 
                   (6, 0.7732081411523537),(7, 0.8280254777070064),(8, 0.8299319727891157),(9, 0.9469026548672567),(10, 0.9540855457227139),(11, 0.9612684365781711),(12, 0.9684513274336283),(13, 0.9756342182890856),(14, 0.9828171091445428),(15, 0.99)]

In [137]:
def generate_bracket(data, estimator, tournament, slots_df=slots):
    """
    Generate a single bracket for the 2024 NCAA tournament.

    Parameters
    ----------
    data : pd.DataFrame
        Regular season data for the 2024 teams competing in the tournament.
    estimator : sklearn estimator
        Pre-trained estimator to use for modeling.
    tournament : str
        'M' or 'W'.
    slots : pd.DataFrame
        Slots for the 2024 tournament.

    Returns
    -------
    result_df : pd.DataFrame
        DataFrame with the predicted outcomes of the tournament.
    
    """

    # get a copies of data to avoid modifying the original
    features = data.copy()
    slots = slots_df.copy()

    # create empty results
    result_df = pd.DataFrame(columns=["Slot", "Team"])

    # define scaler
    scaler = MinMaxScaler()

    # 6 rounds in a single bracket
    for i in range(1, 7):
        # get slots for round
        slots_round = slots[slots['Slot'].str.contains(f'R{i}')].reset_index(drop=True)

        # holds data for each matchup
        round_matchups = []

        # loop through the slots
        for idx, row in slots_round.iterrows():
            # get team A and team B
            A = features[features['FullSeed'] == row['StrongSeed']].reset_index(drop=True)
            B = features[features['FullSeed'] == row['WeakSeed']].reset_index(drop=True)

            # rename cols
            A = A.add_prefix('A_')
            B = B.add_prefix('B_')

            # create matchup dataframe
            combined = pd.concat([A, B], axis=1)

            # append combined row to the list
            round_matchups.append(combined)

        # concatenate all matchup rows into a single DataFrame
        round_df = pd.concat(round_matchups, axis=0).reset_index(drop=True)

        # add some cols that were in training data
        round_df['A_Loc_A'] = 0
        round_df['A_Loc_H'] = 0
        round_df[['round_2', 'round_3', 'round_4', 'round_5', 'round_6']] = 0
        if i == 1:
            pass
        else:
            round_df[f'round_{i}'] = 1

        # add seed win probs
        

        # drop and sort cols
        round_df = round_df.drop(columns=['Season', 'A_Seed', 'B_Seed'])
        round_df = round_df[sorted_cols]

        # define X and reorder cols
        X = round_df.drop(columns=['A_TeamID', 'A_FullSeed', 'B_TeamID', 'B_FullSeed'])

        # scale data
        X = scaler.fit_transform(X)
        
        # predict the outcomes of the round
        preds = estimator.predict(X)

        # replace preds with full seed of winning team
        preds = np.where(preds > 0, round_df['A_FullSeed'], round_df['B_FullSeed'])

        for slot, winner_seed in zip(slots_round['Slot'], preds):
            # save results to result_df
            result_df.loc[len(result_df.index)] = [slot, winner_seed]

        # edit slots df for next round
        if i != 6:
            next_round_slots = slots[slots['Slot'].str.contains(f'R{i+1}')]

            for idx, row in next_round_slots.iterrows():
                # get the teams playing in that slot for the next round
                team1 = result_df[result_df['Slot'] == row['StrongSeed']]['Team'].values[0]
                team2 = result_df[result_df['Slot'] == row['WeakSeed']]['Team'].values[0]

                # update the slots df
                slots.loc[slots['Slot'] == row['Slot'], 'StrongSeed'] = team1
                slots.loc[slots['Slot'] == row['Slot'], 'WeakSeed'] = team2

        # drop teams that have been eliminated
        features = features[features['FullSeed'].isin(result_df['Team'])].reset_index(drop=True)

    # add bracket and tournament cols
    result_df['Bracket'] = 1
    result_df['Tournament'] = tournament

    return result_df


In [ ]:
# train best men's model on entire dataset
mmodel = LogisticRegression(n_jobs=-1, random_state=SEED)

# define X and y
X = mfcomp.drop(columns=['A_TeamID', 'A_FullSeed', 'B_TeamID', 'B_FullSeed', 'score_diff_adj', 'win'])
X = X[sorted(X.columns)]
y = mfcomp['win']

# scale data
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# fit model
mmodel.fit(X, y)

In [ ]:
# train best women's model on entire dataset
wmodel = LogisticRegression(n_jobs=-1, random_state=SEED)

# define X and y
X = wfcomp.drop(columns=['A_TeamID', 'A_FullSeed', 'B_TeamID', 'B_FullSeed', 'A_seed_win_prob', 'B_seed_win_prob', 'score_diff_adj', 'win'])
X = X[sorted(X.columns)]
y = wfcomp['win']

# scale data
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# fit model
mmodel.fit(X, y)

In [ ]:
# num brackets
n_brackets = 100000

# men's
result_m = generate_bracket(data=mdf_2024, estimator=mmodel)

# women's
result_w = generate_bracket(data=wdf_2024, estimator=wmodel)

# combine results
submission = pd.concat([result_m, result_w])
submission = submission.reset_index(drop=True)
submission.index.names = ['RowId']

# reorder
submission = submission[['Tournament', 'Bracket', 'Slot', 'Team']]

In [ ]:
# save
submission.to_csv('submission.csv')
submission.head()